In [0]:
%sql
-----------------------------
-- CREATE FACT SALES TABLE
-----------------------------
CREATE TABLE IF NOT EXISTS real_time_projects.ecommerce_historical.fact_sales (
    order_id STRING,
    order_item_id STRING,
    customer_id STRING,
    product_id STRING,

    order_purchase_timestamp TIMESTAMP,
    order_date DATE,

    price DOUBLE,
    freight_value DOUBLE,
    revenue DOUBLE,

    customer_state STRING,
    product_category_name STRING,

    payment_value DOUBLE,
    order_status STRING,
    load_date DATE
)
USING DELTA;

In [0]:
from pyspark.sql.functions import *
# --------------------
# READ DELTA TABLE
# --------------------
orders_df = spark.table("real_time_projects.ecommerce_historical.orders")
order_items_df = spark.table("real_time_projects.ecommerce_historical.order_items")
payments_df = spark.table("real_time_projects.ecommerce_historical.order_payments")

# -------------------------------
# READ DIMENSION DELTA TABLE
# -------------------------------
dim_customers_df = spark.table("real_time_projects.ecommerce_historical.dim_customer").filter("is_active = 'Y'")
dim_products_df = spark.table("real_time_projects.ecommerce_historical.dim_product").filter("is_active = 'Y'")

# ---------------------------------------------------------
# JOIN ORDER & ORDER ITEMS, ENSURE UNIQUE COLUMN NAMES
# ---------------------------------------------------------
order_items_df = order_items_df.select(
    "order_id",
    col("order_item_id").alias("order_item_id"),
    "product_id",
    "price",
    "freight_value"
)

# ------------------------------
# JOIN ORDER & ORDER ITEMS
# ------------------------------
sales_df = (
    orders_df
    .join(
        order_items_df,
        "order_id",
        "inner"
    ) 
)

# ---------------------------------------
# JOIN CUSTOMER & PRODUCT DIMENSION
# ---------------------------------------
sales_df = (
    sales_df
    .join(dim_customers_df, "customer_id", "left")
    .join(dim_products_df, "product_id", "left")
)

# -----------------
# JOIN PAYMENTS
# -----------------
sales_df = sales_df.join(
    payments_df.groupBy("order_id")
        .agg(sum("payment_value").alias("payment_value")),
    "order_id",
    "left"
)

# sales_df.printSchema()
# --------------------------
# DERIVE FACT TABLE
# --------------------------
fact_sales_df = (
    sales_df
    .select(
        col("order_id"),
        col("order_item_id"),
        col("customer_id"),
        col("product_id"),

        col("order_purchase_timestamp"),
        to_date("order_purchase_timestamp").alias("order_date"),

        col("price"),
        col("freight_value"),
        (col("price") + col("freight_value")).alias("revenue"),

        col("customer_state"),
        col("product_category_name"),

        col("payment_value"),
        col("order_status"),

        current_date().alias("load_date")
    )
)

# fact_sales_df.printSchema()

# ------------------------
# WRITE TO DELTA TABLE
# ------------------------
fact_sales_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("real_time_projects.ecommerce_historical.fact_sales")

In [0]:
from pyspark.sql.functions import col, sum, when, lit

df = spark.table("real_time_projects.ecommerce_historical.fact_sales")

# ---------------------------------------------------------------------
# DISPLAY THE NUMBER OF NULL VALUES IN EACH COLUMN OF THE DATAFRAME
# ---------------------------------------------------------------------
display(
    df.select([
        count(
            when(
                col(c).isNull(),
                c
            )
        ).alias(c)
        for c in df.columns
    ])
)
